##### import libraries

In [2]:
import pandas as pd
import numpy as np

from my_library import normalize_data, calcul_distances, display_dist_matrix

# Hierarchical clustering algorithm implementation AGNES

##### Get the index of the minimum value of the matrix

In [3]:
def np_min_ign_diagonal(z: np.ndarray):
    row_index , col_index = 0, 1
    for i in range(len(z)):
        for j in range(len(z)):
            if i == j:
                continue
            if z[i, j] < z[row_index, col_index]:
                row_index = i
                col_index = j
    return row_index, col_index

# AGNES algorithm

In [4]:
def hierarchical_clustering_algo_agnes(df: pd.DataFrame, options: dict={}):
    # normalize data
    normalize_data(df, None)

    print(f'df :')
    print(df, end='\n\n')
    # l = dict['label']
    labels = df['id'].values
    df = df.drop(columns=['id', 'class'])
    Z= calcul_distances(df, labels)

    while len(Z) > 1:
        display_dist_matrix(labels, Z)
        # get the index of the minimum value of the matrix
        row_index, col_index = np_min_ign_diagonal(Z)

        # update the matrix
        for j in range(len(Z)):
            if j == col_index:
                Z[row_index, j] = 0
            elif j != row_index:
                Z[row_index, j] = min(Z[row_index, j], Z[col_index, j])
                Z[j, row_index] = Z[row_index, j]
        Z = np.delete(Z, col_index, 0)
        Z = np.delete(Z, col_index, 1)

        labels[row_index] = f'{labels[row_index]}-{labels[col_index]}'
        labels = np.delete(labels, col_index, 0)

    print(df)


# Test

In [5]:
data = pd.read_csv('data.csv')
labels = data['id'].values

hierarchical_clustering_algo_agnes(data)

normilize order of "cheveux" :: ['blond' 'brun' 'rousse']
df :
      id     poids    cheveux  taile  vegetarien  class
0  Salim  0.111111  [1, 0, 0]    0.5           0      1
1  Ahmed  0.444444  [1, 0, 0]    1.0           1      0
2   Alex  0.333333  [0, 1, 0]    0.0           1      0
3  Annie  0.555556  [1, 0, 0]    0.0           0      1
4  Emily  0.777778  [0, 0, 1]    0.5           0      1
5    Ali  0.888889  [0, 1, 0]    1.0           0      0
6   John  1.000000  [0, 1, 0]    0.5           0      0
7  Katie  0.000000  [1, 0, 0]    0.0           1      0

distance of each column ::  {'poids': 0.3333333333333333, 'cheveux': 0.0, 'taile': 0.5, 'vegetarien': 1} / 4 -- result = 0.4583333333333333
distance of each column ::  {'poids': 0.2222222222222222, 'cheveux': 0.6666666666666666, 'taile': 0.5, 'vegetarien': 1} / 4 -- result = 0.5972222222222222
distance of each column ::  {'poids': 0.4444444444444445, 'cheveux': 0.0, 'taile': 0.5, 'vegetarien': 0} / 4 -- result = 0.23611111111111

,Salim,Ahmed,Alex,Annie,Emily,Ali,John,Katie
Salim,0.000000,0.458333,0.597222,0.236111,0.333333,0.486111,0.388889,0.402778
Ahmed,0.458333,0.000000,0.444444,0.527778,0.625000,0.527778,0.680556,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.652778,0.638889,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.500000,0.402778,0.388889
Emily,0.333333,0.625000,0.652778,0.347222,0.000000,0.319444,0.222222,0.736111
Ali,0.486111,0.527778,0.638889,0.500000,0.319444,0.000000,0.152778,0.888889
John,0.388889,0.680556,0.541667,0.402778,0.222222,0.152778,0.000000,0.791667
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.888889,0.791667,0.000000


,Salim,Ahmed,Alex,Annie,Emily,Ali-John,Katie
Salim,0.000000,0.458333,0.597222,0.236111,0.333333,0.388889,0.402778
Ahmed,0.458333,0.000000,0.444444,0.527778,0.625000,0.527778,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.652778,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.402778,0.388889
Emily,0.333333,0.625000,0.652778,0.347222,0.000000,0.222222,0.736111
Ali-John,0.388889,0.527778,0.541667,0.402778,0.222222,0.000000,0.791667
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.791667,0.000000


,Salim,Ahmed,Alex,Annie,Emily-Ali-John,Katie
Salim,0.000000,0.458333,0.597222,0.236111,0.333333,0.402778
Ahmed,0.458333,0.000000,0.444444,0.527778,0.527778,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.388889
Emily-Ali-John,0.333333,0.527778,0.541667,0.347222,0.000000,0.736111
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.000000


,Salim-Annie,Ahmed,Alex,Emily-Ali-John,Katie
Salim-Annie,0.000000,0.458333,0.472222,0.333333,0.388889
Ahmed,0.458333,0.000000,0.444444,0.527778,0.361111
Alex,0.472222,0.444444,0.000000,0.541667,0.250000
Emily-Ali-John,0.333333,0.527778,0.541667,0.000000,0.736111
Katie,0.388889,0.361111,0.250000,0.736111,0.000000


,Salim-Annie,Ahmed,Alex-Katie,Emily-Ali-John
Salim-Annie,0.000000,0.458333,0.388889,0.333333
Ahmed,0.458333,0.000000,0.361111,0.527778
Alex-Katie,0.388889,0.361111,0.000000,0.541667
Emily-Ali-John,0.333333,0.527778,0.541667,0.000000


,Salim-Annie-Emily-Ali-John,Ahmed,Alex-Katie
Salim-Annie-Emily-Ali-John,0.000000,0.458333,0.388889
Ahmed,0.458333,0.000000,0.361111
Alex-Katie,0.388889,0.361111,0.000000


,Salim-Annie-Emily-Ali-John,Ahmed-Alex-Katie
Salim-Annie-Emily-Ali-John,0.000000,0.388889
Ahmed-Alex-Katie,0.388889,0.000000


      poids    cheveux  taile  vegetarien
0  0.111111  [1, 0, 0]    0.5           0
1  0.444444  [1, 0, 0]    1.0           1
2  0.333333  [0, 1, 0]    0.0           1
3  0.555556  [1, 0, 0]    0.0           0
4  0.777778  [0, 0, 1]    0.5           0
5  0.888889  [0, 1, 0]    1.0           0
6  1.000000  [0, 1, 0]    0.5           0
7  0.000000  [1, 0, 0]    0.0           1
